In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

image_dir = "/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Images"
mask_dir = "/home/temgar.s/pneumonia_dataset/Pneumonia/Training/Masks"

save_img_dir = "yolo_dataset/images/train"
save_lbl_dir = "yolo_dataset/labels/train"
os.makedirs(save_img_dir, exist_ok=True)
os.makedirs(save_lbl_dir, exist_ok=True)


def get_yolo_bbox_from_contours(mask, w, h, min_box_size=10):
    
    binary_mask = (mask > 0).astype(np.uint8)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    boxes = []
    for cnt in contours:
        x, y, box_w, box_h = cv2.boundingRect(cnt)
        if box_w < min_box_size or box_h < min_box_size:
            continue  
        x_center = (x + box_w / 2) / w
        y_center = (y + box_h / 2) / h
        box_w_norm = box_w / w
        box_h_norm = box_h / h
        boxes.append((0, x_center, y_center, box_w_norm, box_h_norm))
    return boxes

for mask_file in tqdm(sorted(os.listdir(mask_dir)), desc="Processing masks"):
    if not mask_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    mask_path = os.path.join(mask_dir, mask_file)
    img_path = os.path.join(image_dir, mask_file)

    mask = cv2.imread(mask_path, 0)  # grayscale
    img = cv2.imread(img_path)         # color image

    if mask is None:
        print(f"Warning: Failed to load mask: {mask_path}")
        continue
    if img is None:
        print(f"Warning: Failed to load image: {img_path}")
        continue


    h, w = mask.shape

    boxes = get_yolo_bbox_from_contours(mask, w, h)

    base_name = os.path.splitext(mask_file)[0]
    label_file = os.path.join(save_lbl_dir, base_name + '.txt')
    
    with open(label_file, "w") as f:
        for box in boxes:
            f.write(" ".join(map(str, box)) + "\n")

    img_save_path = os.path.join(save_img_dir, mask_file)
    success = cv2.imwrite(img_save_path, img)
    if not success:
        print(f"Error saving image: {img_save_path}")

print(" YOLO bounding box conversion complete.")

Processing masks:  71%|███████   | 18946/26684 [55:41<18:34,  6.94it/s]

Processing masks:  79%|███████▉  | 21080/26684 [1:01:27<20:58,  4.45it/s]libpng error: Read Error


Processing masks:  90%|████████▉ | 23911/26684 [1:09:12<06:10,  7.48it/s]

Processing masks:  91%|█████████ | 24201/26684 [1:09:58<04:19,  9.56it/s]

Processing masks: 100%|██████████| 26684/26684 [1:19:14<00:00,  5.61it/s]

 YOLO bounding box conversion complete.
